# QML4EO-tutorial: Hybrid Quantum Convolutional Neural Network Classifier
## This is a qiskit adapted version of [QML4EO-tutorial](https://github.com/alessandrosebastianelli/QML4EO-tutorial/tree/main)

Please refer to the following articles to get more insight about this topic:

- [1] Sebastianelli, A., Zaidenberg, D. A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021). On circuit-based hybrid quantum neural networks for remote sensing imagery classification. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 15, 565-580.
- [2] Zaidenberg, D. A., Sebastianelli, A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021, July). Advantages and bottlenecks of quantum machine learning for remote sensing. In 2021 IEEE International Geoscience and Remote Sensing Symposium IGARSS (pp. 5680-5683). IEEE.
- [3] Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
- [4] Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.
- [Qiskit web-page](https://qiskit.org/documentation/machine-learning/tutorials/index.html)
- [Pennylae web-page](https://pennylane.ai/qml/demos_qml.html)

# 1. Clone the base code

Basic functions are wrapped in seprated files to keep the notebook clean. Feel free to explore the code.

In [ ]:
import sys
!rm -r /content/qml4eo_tutorial_qiskit
!git clone https://github.com/alessandrosebastianelli/qml4eo_tutorial_qiskit.git
if '/content/qml4eo_tutorial_qiskit' not in sys.path: sys.path.append('/content/qml4eo_tutorial_qiskit')

# 2. Install missing requirements

AI-related

In [ ]:
!pip install -r /content/qml4eo_tutorial_qiskit/requirements.txt -q

EO-related

In [ ]:
!pip install -q --upgrade pyosv

QC-related

In [ ]:
!pip install -q qiskit-machine-learning==0.6.0

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import numpy as np
import os

from hybrid_model import *
from torch_loader import EuroSATDataModule

torch.multiprocessing.set_start_method('spawn')  
torch.set_float32_matmul_precision('high')

# 3. Download the dataset

In this study, we address the challenge of land use and land cover classification using Sentinel-2 satellite images. The Sentinel-2 satellite images are openly and freely accessible provided in the Earth observation program Copernicus. We present a novel dataset based on Sentinel-2 satellite images covering 13 spectral bands and consisting out of 10 classes with in total 27,000 labeled and geo-referenced images. We provide benchmarks for this novel dataset with its spectral bands using state-of-the-art deep Convolutional Neural Network (CNNs). With the proposed novel dataset, we achieved an overall classification accuracy of 98.57%. The resulting classification system opens a gate towards a number of Earth observation applications. We demonstrate how this classification system can be used for detecting land use and land cover changes and how it can assist in improving geographical maps.

Get more information [here](https://github.com/phelber/EuroSAT).

![](https://github.com/phelber/EuroSAT/blob/master/eurosat_overview_small.jpg?raw=true)

The following snippet will download the EuroSAT dataset and split it in train/validation/test sub-datasets.

In [ ]:
!bash /content/qml4eo_tutorial_qiskit/download_data.sh

# 4. Load data

In [ ]:

# This is the full list of classes in EuroSAT
classes = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop', 'Residential', 'River', 'SeaLake']

# For simplicty we will reduce them to 2
classes = ['AnnualCrop', 'Forest']

# Torch lightning data loader module
data_module = EuroSATDataModule(
        whitelist_classes = classes, # Here we are selecting the classes to be classified
        root_dir          = '/content',      
        batch_size        = 16, 
        bands             = [3,2,1], # Here we are selecting only the RGB bands
        num_workers       = 2, # This depends on your hardware, google colab has only 2
)

## 4.1 Plot some data

In [ ]:
data_module.setup()
loader = data_module.train_dataloader()
x, y = next(iter(loader))

fig, axes = plt.subplots(nrows=4, ncols=4)
axes = axes.flatten()

if 'cpu' in str(x.device): x = x.numpy()
else: x = x.cpu().detach().numpy()

x = np.moveaxis(x, 1, -1)


for i, ax in enumerate(axes):
    ax.imshow(x[i,...])
    ax.axis(False)

fig.tight_layout()
plt.show()


# 5. Run Hybrid model

![](https://www.researchgate.net/publication/354779281/figure/fig6/AS:11431281080884121@1661423072104/Proposed-hybrid-QNN-classifier-1-The-network-is-a-modified-version-of-LeNet-5-where-the.png)


The method proposed in [1] introduces a **hybrid quantum-classical convolutional neural network (QCNN)** designed for remote sensing image classification, specifically applied to land-use and land-cover (LULC) classification using the EuroSAT dataset. 

The architecture integrates a quantum circuit layer into a classical CNN (based on a modified LeNet-5), with quantum layers implemented using different types of gate-based circuits, including both entangled and non-entangled designs. Classical data are first embedded into quantum states via unitary transformations, enabling the quantum circuit to process them through parametrized gates.

With respect to [1], here we are going to use a more "complex" classical deep learning network.

## 5.1 Torch lightning Hybrid AlexNet

In [ ]:
number_of_qubits = 2 # Feel free to "play" with this parameter
network = LightningAlexNetModule(n_qubits=number_of_qubits, num_classes=len(classes))

## 5.2 Train and monitor the hybrid model

In [ ]:
log_name  = 'hybrid_model'

# will log some relevant information about training/validation/testing of the model
tb_logger = pl.loggers.TensorBoardLogger(os.path.join('lightning_logs','classifiers'), name=log_name)

# will save models weights
checkpoint_callback = ModelCheckpoint(
    dirpath         = os.path.join('saved_models','classifiers'),
    filename        = log_name,
    monitor         = 'val_loss',
    save_top_k      = 1,
    mode            = 'min',
)

# Instantiate Trainer
trainer = pl.Trainer(max_epochs=30, callbacks=[checkpoint_callback], logger=tb_logger)

# Train the model
trainer.fit(network, data_module)